After familiarizing yourself with the syntax of spannerlog, <br>
let's explore another example that highlights the power of this declarative language. <br>
given a story.txt file which contains in between its lines parent-child information, the information is provided as follows: <br>
1- X is the parent/father of Y <br>
2- Y is the child/daughter of X

The story also has some last names.
The task is to identify the last name of all individuals.
It's important to note that you don't have access to individual last names, but you can deduce them by constructing family trees based on the information provided in the story.

Here is the first line of the story as an example:
```plaintext
Once upon a time, there was a little girl named Aria Walker. Aria is the daughter of Owen.
```
What you should deduce from this line is that Owen is the parent of Aria as it matches the first template of provided information (Y is the daughter of X), <br>
and since Aria's name was given along with her lastname, <br>
you should also conclude that Owen's last name is also Walker.

Take a couple of minutes to think a solution for this problem.
---

Now that you understand the complexity of deducing last names based on family trees and extracting the parent-child relationships using Python, <br>
let us show you how easily we can do that by incorporating spannerlog into the equation.


In [ ]:
from spannerlib import get_magic_session,Span

sess = get_magic_session()

story = Span("""
Once upon a time, there was a little girl named Aria Walker. Aria is the daughter of Owen. Owen was a kind and caring father who adored his daughter's laughter and curiosity.
In the neighboring town, there lived a cheerful boy named Eliana. Eliana's eyes sparkled with innocence, Owen is the father of Eliana.
Owen cherished every moment spent with Eliana, teaching her valuable life lessons and nurturing her dreams.
One sunny day, Aria gave birth to a wonderful girl called Mila. Mila is the daughter of Aria, Aria, watched over her with unwavering love and pride.
Aria and Mila formed an instant bond, becoming the best of friends and had a blossoming friendship.

As the years passed, Mila grew into a confident young girl, thanks to the support and encouragement of her mother, Aria. Mila, in turn,
became a caring sibling to her little brother,Owen is the father of Caleb. Their bond was unbreakable,
and they shared countless adventures together under the watchful eyes of their mother.

Across the village, there lived a young girl named Lila. Lila was a spirited and curious child, Lila is the daughter of Benjamin, she was
always seeking new adventures in the world around her. 
Her father, Benjamin Smith, was a wise and gentle man who cherished his daughter's laughter and inquisitive nature.
Benjamin had another child, Daniel is the child of Benjamin, Yaer is the parent of Benjamin.
""",name='story')

sess.import_var("story", story)

In [ ]:
%%spannerlog
parent(X,Y) <- rgx("(\w+)\s+is\s+the\s+daughter\s+of\s+(\w+)",$story) -> (Y,X)
parent(X,Y) <- rgx("(\w+)\s+is\s+the\s+child\s+of\s+(\w+)",$story) -> (Y,X)
parent(X,Y) <- rgx("(\w+)\s+is\s+the\s+parent\s+of\s+(\w+)",$story) -> (X,Y)
parent(X,Y) <- rgx("(\w+)\s+is\s+the\s+father\s+of\s+(\w+)",$story) -> (X,Y)
lastname(X,Y)<-rgx("([A-Z][a-z]+)\s([A-Z][a-z]+)",$story) -> (X,Y)
?parent(X,Y)
?lastname(X,Y)

'?parent(X,Y)'

,X,Y
0,"[@story,86,90) ""Owen""","[@story,62,66) ""Aria"""
1,"[@story,281,285) ""Owen""","[@story,303,309) ""Eliana"""
2,"[@story,507,511) ""Aria""","[@story,483,487) ""Mila"""
3,"[@story,851,855) ""Owen""","[@story,873,878) ""Caleb"""
4,"[@story,1116,1124) ""Benjamin""","[@story,1092,1096) ""Lila"""
5,"[@story,1357,1365) ""Benjamin""","[@story,1334,1340) ""Daniel"""
6,"[@story,1367,1371) ""Yaer""","[@story,1389,1397) ""Benjamin"""


'?lastname(X,Y)'

,X,Y
0,"[@story,49,53) ""Aria""","[@story,54,60) ""Walker"""
1,"[@story,1202,1210) ""Benjamin""","[@story,1211,1216) ""Smith"""


## Deducing Last Names Based on Family Trees with spannerlog

The following spannerlog code provides a simple yet powerful way to deduce the last name of each person based on the family tree information extracted to the parent relation.

In [ ]:
%%spannerlog
Family(X,Y) <- lastname(Y,X)
Family(X,Y) <- Family(X,Z), parent(Y,Z)
Family(X,Y) <- Family(X,Z), parent(Z,Y)
?Family(X,Y)

'?Family(X,Y)'

,X,Y
0,"[@story,54,60) ""Walker""","[@story,49,53) ""Aria"""
1,"[@story,1211,1216) ""Smith""","[@story,1202,1210) ""Benjamin"""
